# CNN Stacking Model

This notebook is used to train a CNN model for All class classification on top of the stacking images.

### Importing Required Libraries

This section will import the required libaries that will be used to actually implement the training for the Vanilla RNN Binary Classification Model

In [24]:
import numpy as np
import pandas as pd
import os
import glob
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, BatchNormalization, Flatten, Dense
from keras.preprocessing import sequence
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import time

Adding this so that can use the python scripts for loading the training data

In [2]:
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

Importing the python script training data loader function. This function loads the data from the *.wav* files directly

In [3]:
from py_scripts.directory_funcs import *
from py_scripts.wav_file_funcs import *
from py_scripts.misc_audio_signal_funcs import *
from py_scripts.raw_training_data_creation import load_irmas_data

### Loading Training Data

This section defines the training data generators the script from the training data creation module to load the training data.

In [4]:
# First define the training data directory, and find the different classes that exist
training_data_dir = '../../data/images_jpg/train/'
validation_data_dir = '../../data/images_jpg/validation/'
test_data_dir = '../../data/images_jpg/test/'
get_subdirectory_names(training_data_dir)

['cel', 'cla', 'flu', 'gac', 'gel', 'org', 'pia', 'sax', 'tru', 'vio', 'voi']

Define the generators for each category (Train, Validation, Test)

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory('../../data/images_jpg/train/', target_size=(150, 150),
                                                    batch_size=64, class_mode='categorical')
validaton_generator = validation_datagen.flow_from_directory('../../data/images_jpg/validation/', target_size=(150, 150),
                                                             batch_size=64, class_mode='categorical')
test_generator = test_datagen.flow_from_directory('../../data/images_jpg/test/', target_size=(150, 150),
                                                  batch_size=64, class_mode='categorical')

Found 4736 images belonging to 5 classes.
Found 1013 images belonging to 5 classes.
Found 1013 images belonging to 5 classes.


# Defining the Model

In this section, we define the CNN model for the stacked images

In [29]:
model = Sequential()
model.add(Conv2D(16, (5, 5), strides=(2, 2), activation='relu', input_shape = (150, 150, 3)))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(32, (5, 5), strides = (1, 1), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Conv2D(32, (5, 5), strides = (1, 1), activation='relu'))
model.add(MaxPool2D((2, 2)))
model.add(Flatten())
model.add(Dense(5, activation = 'softmax'))
# Compiling the model
model.compile(loss= 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_40 (Conv2D)           (None, 73, 73, 16)        1216      
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 36, 36, 16)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 32, 32, 32)        12832     
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 12, 12, 32)        25632     
_________________________________________________________________
max_pooling2d_37 (MaxPooling (None, 6, 6, 32)          0         
_________________________________________________________________
flatten_12 (Flatten)         (None, 1152)              0         
__________

In [31]:
# Adding a checkpoint
parent_weight_save_dir = '../../data/Training Results/CNN_Stacking/Weights'
tensor_board_dir_path = '../../data/Training Results/CNN_Stacking/TensorBoard'
check_output_directory(parent_weight_save_dir)
current_experiment_name = f'CT-{time.time()}'
weight_file_path = os.path.join(parent_weight_save_dir, f'{current_experiment_name}.hdf5')
tensor_board_file_path = os.path.join(tensor_board_dir_path, current_experiment_name)
check_output_directory(tensor_board_file_path)
checkpoint = ModelCheckpoint(weight_file_path, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
tensorboard = TensorBoard(log_dir=tensor_board_file_path)
early_stopping_criteria = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=0, mode='auto')
callbacks_list = [tensorboard, checkpoint, early_stopping_criteria]

In [ ]:
history = model.fit_generator(train_generator, epochs=50, validation_data=validaton_generator, callbacks=callbacks_list)

Epoch 1/50
74/74 [==============================] - 191s 3s/step - loss: 1.5049 - acc: 0.3332 - val_loss: 1.4183 - val_acc: 0.3771

Epoch 00001: val_acc improved from -inf to 0.37710, saving model to ../../data/Training Results/CNN_Stacking/Weights\CT-1544941011.688695.hdf5
Epoch 2/50
74/74 [==============================] - 139s 2s/step - loss: 1.4292 - acc: 0.3670 - val_loss: 1.3552 - val_acc: 0.3988

Epoch 00002: val_acc improved from 0.37710 to 0.39882, saving model to ../../data/Training Results/CNN_Stacking/Weights\CT-1544941011.688695.hdf5
Epoch 3/50
74/74 [==============================] - 139s 2s/step - loss: 1.3838 - acc: 0.3999 - val_loss: 1.4133 - val_acc: 0.3929

Epoch 00003: val_acc did not improve from 0.39882
Epoch 4/50
74/74 [==============================] - 134s 2s/step - loss: 1.3446 - acc: 0.4088 - val_loss: 1.3307 - val_acc: 0.4235

Epoch 00004: val_acc improved from 0.39882 to 0.42349, saving model to ../../data/Training Results/CNN_Stacking/Weights\CT-1544941011